In [ ]:
#Defining the model

model = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
output_feat=args.output_size,seq_len=args.obs_seq_len,
kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)


#Training settings

optimizer = optim.SGD(model.parameters(),lr=args.lr)

if args.use_lrschd:
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.1)



checkpoint_dir = './checkpoint/'+args.tag+'/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)



print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

#Training
metrics = {'train_loss':[],  'val_loss':[]}
constant_metrics = {'min_val_epoch':-1, 'min_val_loss':9999999999999999}

def train(epoch):
    global metrics,loader_train
    model.train()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_train)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1


    for cnt,batch in enumerate(loader_train):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        optimizer.zero_grad()
        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)



        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            loss.backward()

            if args.clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip_grad)


            optimizer.step()
            #Metrics
            loss_batch += loss.item()
            print('TRAIN:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['train_loss'].append(loss_batch/batch_count)




def vald(epoch):
    global metrics,loader_val,constant_metrics
    model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1

    for cnt,batch in enumerate(loader_val):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            #Metrics
            loss_batch += loss.item()
            print('VALD:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['val_loss'].append(loss_batch/batch_count)

    if  metrics['val_loss'][-1]< constant_metrics['min_val_loss']:
        constant_metrics['min_val_loss'] =  metrics['val_loss'][-1]
        constant_metrics['min_val_epoch'] = epoch
        torch.save(model.state_dict(),checkpoint_dir+'val_best.pth')  # OK
        check_test_performance()

print('Training started ...')
for epoch in range(args.num_epochs):
    train(epoch)
    vald(epoch)
    if args.use_lrschd:
        scheduler.step()


    print('*'*30)
    print('Epoch:',args.tag,":", epoch)
    for k,v in metrics.items():
        if len(v)>0:
            print(k,v[-1])


    print(constant_metrics)
    print('*'*30)

    with open(checkpoint_dir+'metrics.pkl', 'wb') as fp:
        pickle.dump(metrics, fp)

    with open(checkpoint_dir+'constant_metrics.pkl', 'wb') as fp:
        pickle.dump(constant_metrics, fp)



Data and model loaded
Checkpoint dir: ./checkpoint/social-tag/
Training started ...


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


TRAIN: 	 Epoch: 0 	 Loss: 0.01595938391983509
TRAIN: 	 Epoch: 0 	 Loss: 0.01579183293506503
TRAIN: 	 Epoch: 0 	 Loss: 0.01562640815973282
TRAIN: 	 Epoch: 0 	 Loss: 0.01544566941447556
TRAIN: 	 Epoch: 0 	 Loss: 0.01526978500187397


 26%|██▌       | 238/921 [58:45<2:48:37, 14.81s/it]


TRAIN: 	 Epoch: 0 	 Loss: 0.015112685815741619
TRAIN: 	 Epoch: 0 	 Loss: 0.014960332374487604
TRAIN: 	 Epoch: 0 	 Loss: 0.014812218374572694
TRAIN: 	 Epoch: 0 	 Loss: 0.01466385368257761
TRAIN: 	 Epoch: 0 	 Loss: 0.014520969148725272
TRAIN: 	 Epoch: 0 	 Loss: 0.014382676136764612
TRAIN: 	 Epoch: 0 	 Loss: 0.014241795986890793
TRAIN: 	 Epoch: 0 	 Loss: 0.014097469572264414
TRAIN: 	 Epoch: 0 	 Loss: 0.013951535802334547
TRAIN: 	 Epoch: 0 	 Loss: 0.013794143497943879
TRAIN: 	 Epoch: 0 	 Loss: 0.013640973484143615
TRAIN: 	 Epoch: 0 	 Loss: 0.013556004642988697
VALD: 	 Epoch: 0 	 Loss: 0.01103119645267725
VALD: 	 Epoch: 0 	 Loss: 0.010940011125057936
VALD: 	 Epoch: 0 	 Loss: 0.013402903142074743
VALD: 	 Epoch: 0 	 Loss: 0.012757524282870416
**************************************************
Number of samples: 20
**************************************************
Model being tested are: ['/kaggle/working/checkpoint/social-tag']
**************************************************
Evaluating mo

...

Testing ....


ADE: 0.34134521070364837  FDE: 0.5335576055776223
**************************************************
******************************
Epoch: social-tag : 182
train_loss -0.012148792149894165
val_loss -0.011532564125137177
{'min_val_epoch': 182, 'min_val_loss': -0.011532564125137177}
******************************
TRAIN: 	 Epoch: 183 	 Loss: -0.011596936732530594
TRAIN: 	 Epoch: 183 	 Loss: -0.012811702210456133
TRAIN: 	 Epoch: 183 	 Loss: -0.012807559532423815
TRAIN: 	 Epoch: 183 	 Loss: -0.012442910112440586
TRAIN: 	 Epoch: 183 	 Loss: -0.012270738184452058
TRAIN: 	 Epoch: 183 	 Loss: -0.012229831423610449
TRAIN: 	 Epoch: 183 	 Loss: -0.012317716277071409
TRAIN: 	 Epoch: 183 	 Loss: -0.012313807616010308
TRAIN: 	 Epoch: 183 	 Loss: -0.012217713416450553
TRAIN: 	 Epoch: 183 	 Loss: -0.012246084492653608
TRAIN: 	 Epoch: 183 	 Loss: -0.012265288609672676
TRAIN: 	 Epoch: 183 	 Loss: -0.01219557086005807
TRAIN: 	 Epoch: 183 	 Loss: -0.012185580312059475
TRAIN: 	 Epoch: 183 	 Loss: -0.0121785

100%|██████████| 921/921 [01:34<00:00,  9.70it/s]

Testing ....


ADE: 0.34325901825389826  FDE: 0.5560145048561989
**************************************************
******************************
Epoch: social-tag : 211
train_loss -0.012243538445821314
val_loss -0.011872185918385398
{'min_val_epoch': 211, 'min_val_loss': -0.011872185918385398}
******************************
TRAIN: 	 Epoch: 212 	 Loss: -0.01174123864620924
TRAIN: 	 Epoch: 212 	 Loss: -0.012139569967985153
TRAIN: 	 Epoch: 212 	 Loss: -0.012118939310312271
TRAIN: 	 Epoch: 212 	 Loss: -0.012288224417716265
TRAIN: 	 Epoch: 212 	 Loss: -0.012338905222713947
TRAIN: 	 Epoch: 212 	 Loss: -0.012388760223984718
TRAIN: 	 Epoch: 212 	 Loss: -0.012461910689515727
TRAIN: 	 Epoch: 212 	 Loss: -0.01229353272356093
TRAIN: 	 Epoch: 212 	 Loss: -0.01221213500118918
TRAIN: 	 Epoch: 212 	 Loss: -0.012292828410863876
TRAIN: 	 Epoch: 212 	 Loss: -0.012323652986775745
TRAIN: 	 Epoch: 212 	 Loss: -0.012294557799274722
TRAIN: 	 Epoch: 212 	 Loss: -0.012282741685899405
TRAIN: 	 Epoch: 212 	 Loss: -0.012297202